In [44]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [31]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.ImageFolder(root='output_resized/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

testset = torchvision.datasets.ImageFolder(root='output_resized/valid', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True)

classes = ('Akhil', 'Ceepon', 'Gurpreet', 'Murali','Nikhil', 'Subhani')

In [32]:
num_classes = 6
net = torchvision.models.resnet34(pretrained=True)

num_ftrs = net.fc.in_features

net.fc = nn.Linear(num_ftrs, num_classes)
net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to C:\Users\Subhani/.torch\models\resnet34-333f7ec4.pth
87306240it [01:40, 868805.31it/s] 


In [33]:
for epoch in range(25):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
print('Finished Training')

[1,   100] loss: 0.694
[1,   200] loss: 0.209
[1,   300] loss: 0.288
[2,   100] loss: 0.136
[2,   200] loss: 0.116
[2,   300] loss: 0.130
[3,   100] loss: 0.038
[3,   200] loss: 0.070
[3,   300] loss: 0.096
[4,   100] loss: 0.085
[4,   200] loss: 0.050
[4,   300] loss: 0.023
[5,   100] loss: 0.027
[5,   200] loss: 0.037
[5,   300] loss: 0.041
[6,   100] loss: 0.017
[6,   200] loss: 0.042
[6,   300] loss: 0.006
[7,   100] loss: 0.015
[7,   200] loss: 0.039
[7,   300] loss: 0.025
[8,   100] loss: 0.032
[8,   200] loss: 0.100
[8,   300] loss: 0.014
[9,   100] loss: 0.029
[9,   200] loss: 0.004
[9,   300] loss: 0.003
[10,   100] loss: 0.055
[10,   200] loss: 0.042
[10,   300] loss: 0.012
[11,   100] loss: 0.033
[11,   200] loss: 0.029
[11,   300] loss: 0.063
[12,   100] loss: 0.026
[12,   200] loss: 0.017
[12,   300] loss: 0.007
[13,   100] loss: 0.015
[13,   200] loss: 0.004
[13,   300] loss: 0.002
[14,   100] loss: 0.003
[14,   200] loss: 0.021
[14,   300] loss: 0.048
[15,   100] loss: 0

In [34]:
PATH = './resnet34.pth'
torch.save(net.state_dict(), PATH)

In [35]:
dataiter = iter(testloader)
images, labels = dataiter.next()
# imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))
images, labels = images.cuda(), labels.cuda()

GroundTruth:  Gurpreet Gurpreet Subhani Subhani


In [36]:
outputs = net(images)

In [37]:
outputs.shape

torch.Size([4, 6])

In [38]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

Predicted:  Gurpreet Gurpreet Subhani Subhani


In [40]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 97 %
